This script reads and saves all US POI data

In [1]:
import requests
import pandas as pd
import sys
from io import BytesIO
import gzip
from tqdm import tqdm
import io
import os
from global_variables import *

In [2]:
poi_csv_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/all_us_poi_with_store_id.csv'

In [3]:
results = requests.get(url=POI_Data_PRODUCT_API_PATH,
                       params={}, # optionally set date value here
                       headers={'X-API-KEY': API_KEY,
                                'accept': 'application/json'
                               })
response_json = results.json()

In [4]:
with tqdm(total=len(response_json['download_links']) ) as pbar:
    for link_data in response_json['download_links']:
    
        pbar.update(1)
    
        df = pd.read_csv(BytesIO(requests.get(link_data['link']).content), compression="gzip")
        df = df[df['ISO_COUNTRY_CODE'] == 'US'][['CATEGORY_TAGS', 'CITY', 'LATITUDE','LONGITUDE','NAICS_CODE','PLACEKEY','POSTAL_CODE','REGION','STREET_ADDRESS','SUB_CATEGORY','TOP_CATEGORY','LOCATION_NAME','STORE_ID','WEBSITE']]
        # df = df[(df['TOP_CATEGORY'] == 'Depository Credit Intermediation') | (df['NAICS_CODE'].astype(str).str[:3] == '522')]
        df = df.reset_index()
    
        if os.path.isfile(poi_csv_path):
            header_option = False  
        else:
            header_option = True  
    
        df.to_csv(poi_csv_path, mode='a', header=header_option, index=False)

df = pd.read_csv(poi_csv_path)    

with gzip.open(poi_csv_path+".gz", 'wt', encoding='utf-8') as gzipped_file:
    df.to_csv(gzipped_file, index=False)

100%|██████████| 64/64 [32:05<00:00, 30.09s/it]
